In [1]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [2]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [4]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 16813124 (64.14 MB)
Trainable params: 16813124 (64.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
conv_base.trainable = False

In [7]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='../Dataset/Alzheimer/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='training'  # Use this subset for training
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='../Dataset/Alzheimer/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='validation'  # Use this subset for validation
)

Found 5121 files belonging to 4 classes.
Found 1279 files belonging to 4 classes.


In [8]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(train_ds,epochs=15,validation_data=validation_ds)

Epoch 1/15


161/161 [==============================] - 493s 3s/step - loss: 1.0029 - accuracy: 0.5723 - val_loss: 1.2362 - val_accuracy: 0.5020
Epoch 2/15
161/161 [==============================] - 364s 2s/step - loss: 0.7765 - accuracy: 0.6717 - val_loss: 1.2301 - val_accuracy: 0.4871
Epoch 3/15
161/161 [==============================] - 359s 2s/step - loss: 0.6563 - accuracy: 0.7211 - val_loss: 1.0494 - val_accuracy: 0.5895
Epoch 4/15
161/161 [==============================] - 389s 2s/step - loss: 0.5149 - accuracy: 0.7922 - val_loss: 0.9834 - val_accuracy: 0.6208
Epoch 5/15
161/161 [==============================] - 375s 2s/step - loss: 0.4857 - accuracy: 0.8039 - val_loss: 0.8763 - val_accuracy: 0.6458
Epoch 6/15
161/161 [==============================] - 362s 2s/step - loss: 0.4081 - accuracy: 0.8323 - val_loss: 0.9492 - val_accuracy: 0.6372
Epoch 7/15
161/161 [==============================] - 351s 2s/step - loss: 0.3511 - accuracy: 0.8617 - val_loss: 1.0690 - val_accuracy: 0.65